<a href="https://colab.research.google.com/github/STEM-tisch/TensorFlow-Lite/blob/master/3.%20Fashion%20MNIST%20with%20TFLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>